In [ ]:
import os
# Specify the new directory path
parent_directory = 'D:/Projects/CornYield/src'

# Change the current directory
os.chdir(parent_directory) 
# %pwd

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from dataset import get_filename_metadata
from dl.dl_dataset import MixedDataset
from dl.train import train_with_cross_validation, train, validate, data_transform

In [ ]:
from dl.model import ResNetRegression

In [ ]:
crop_var={
    3: 'all',
    2: 'P9998',
    1: 'CH 192-10',
    0: 'DKC 51-91'
}

irrigate_var={
    2: 'All',
    0: 'Full',
    1: 'Deficit'
}
yield_file = 'D:/Corn_Yield/BL2022_Yld_label.csv'
weather_file = 'D:/Corn_Yield/CoAgMet ET_VBA_calc_2022.xlsm'


In [ ]:
img_path_list=[
    'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220616_DOY167_extracted_filled',
    'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220628_DOY179_extracted_filled',
    'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220708_DOY189_extracted_filled',
    'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220720_DOY201_extracted_filled'
]
doy_list=[167, 179, 189, 201]

In [ ]:

# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220616_DOY167_extracted_filled'
# out_path = '../output/DOY167/'
# doy = 167
img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220705_DOY186_extracted_filled'
out_path = '../output/DOY186/'
doy = 186
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220720_DOY201_extracted_filled'
# out_path = '../output/DOY201/'
# doy=201


In [ ]:
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220708_DOY189_extracted_filled'
# out_path = '../output/DOY189/'
# doy=189

In [ ]:
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220628_DOY179_extracted_filled'
# out_path = '../output/DOY179/'
# doy = 179

In [ ]:
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220716_DOY197_extracted_filled'
# out_path = '../output/DOY197/'
# doy = 197

In [ ]:
def create_metadata(yield_pf, weather_file, doy):

    weather_pf = pd.read_excel(weather_file,sheet_name=1, header=[0,1,3])
    # header_name = weather_pf.columns
    # selected_rows = weather_pf[(weather_pf[header_name[-1]] > doy-6) & (weather_pf[header_name[-1]] < doy+6)]
    weather_pf = pd.read_excel(weather_file,sheet_name=1, header=[0,1,3])
    weather_pf = weather_pf.dropna()

    selected_rows = weather_pf[(weather_pf.iloc[:,-1] > doy-6) & (weather_pf.iloc[:,-1]  < doy+6) ]

    air_temperature = selected_rows.iloc[:,2].mean()
    precipitation = selected_rows.iloc[:,9].mean()
    soil_temperature = selected_rows.iloc[:,10].mean()
    soil_temperature2 = selected_rows.iloc[:,11].mean()

    metadata = pd.DataFrame(index=range(len(yield_pf)))
    metadata['Variety'] = yield_pf['Variety_int']/2
    metadata['Irrigation'] = yield_pf['Irrigation_int']/1
    metadata['DOY'] = doy / 366
    metadata['Month'] = (doy / 30)/12
    metadata['Stage'] = (doy/30 - 5)/5
    metadata['air_temperature'] = air_temperature
    metadata['precipitation'] = precipitation
    metadata['soil_temperature'] = soil_temperature
    metadata['soil_temperature2'] = soil_temperature2
    return metadata

CROP_Variety = {
    'P9998': 2,
    'CH 192-10':1,
    'DKC 51-91':0
}


In [ ]:
def get_multi_data(img_path_list, yield_file,keyword,doy_list, analyze_variety_id, analyze_irrigation_id:int=2):
    all_img_list = []
    all_yield_list = []
    meta_df = pd.DataFrame()
    # all_train_indices = []
    yield_pf_all = pd.DataFrame()
    for (img_path,doy) in zip(img_path_list,doy_list):
        print(doy)
        img_list, yield_pf = get_filename_metadata(img_path, yield_file, keyword)

        if analyze_variety_id != 3 or analyze_irrigation_id != 2:
            if analyze_variety_id != 3 and analyze_irrigation_id != 2:
                yield_pf = yield_pf[(yield_pf['Variety_int'] == analyze_variety_id) & (yield_pf['Irrigation_int'] == analyze_irrigation_id)]
            elif analyze_variety_id != 3:
                yield_pf = yield_pf[yield_pf['Variety_int'] == analyze_variety_id]
            elif  analyze_irrigation_id != 2:
                yield_pf = yield_pf[ yield_pf['Irrigation_int'] == analyze_irrigation_id]
                
            indices = yield_pf.index.tolist()
            img_list = [img_list[i] for i in indices]
            #### yield_pf = yield_pf[yield_pf['Variety_int'] == analyze_variety_id & yield_pf['Irrigation_int'] == analyze_irrigation_id]
            yield_pf = yield_pf.reset_index(drop=True)

        yield_list = list(yield_pf['Yield_Bu_Ac'])

        '''get metadata'''
        metadata = create_metadata(yield_pf, weather_file,doy)


        meta_df = pd.concat([meta_df, metadata], ignore_index=True)
        yield_pf_all = pd.concat([yield_pf_all, yield_pf], ignore_index=True)
        all_img_list.extend(img_list)
        all_yield_list.extend(yield_list)

    return all_img_list, all_yield_list, meta_df, yield_pf_all

In [ ]:
key_word_list = ['Ref_filled.tif']
analyze_variety_id = 3
analyze_irrigation_id = 2
train_col='TRAIN_70'
for keyword in key_word_list:

    
    # img_list, yield_list, metadata, yield_pf  = get_multi_data(img_path_list, yield_file,keyword,doy_list, analyze_variety_id, analyze_irrigation_id)

    img_list, yield_list, metadata, yield_pf = get_multi_data([img_path], yield_file,keyword,[doy], analyze_variety_id, analyze_irrigation_id)

    # VI_list = ['evi', 'ndvi', 'ndre', 'gndvi']
    # VI_list = ['evi', 'ndvi']
    # suffix_list = ['LWIR_filled.tif']
# 
    VI_list = None
    suffix_list = None
    # print(metadata)
    # train_val_dataset = MixedDataset(img_list, yield_list, metadata)

        # Use train_test_split to split the indices into training and testing sets
    # train_indices, test_indices = train_test_split(range(len(img_list)), test_size=test_size, random_state=39)
    train_indices = list(yield_pf[yield_pf[train_col] == 1].index)
    test_indices = list(yield_pf[yield_pf[train_col] == 0].index)

    print(len(train_indices), len(test_indices))


    train_val_dataset = MixedDataset([img_list[i] for i in train_indices], [yield_list[i] for i in train_indices], metadata.loc[train_indices], VI_list=VI_list, suffix_list=suffix_list,  transform=data_transform())
    test_dataset = MixedDataset([img_list[i] for i in test_indices], [yield_list[i] for i in test_indices], metadata.loc[test_indices], suffix_list=suffix_list, VI_list=VI_list)



2. Extract Features

In [ ]:
import torch
from dl.model import ResNetFNN, ResNetFNN_V2

In [ ]:
import torch.optim as optim

in_channel = 5
num_epochs = 150
batch_size = 32

# Initialize an empty list to store fold-wise performance
fold_accuracies = []
resname='resnet18'
# Initialize a new model for each fold
# model = CNNRegression(in_channel)
# model = ResNetFNN(in_channel,9,1)
model = ResNetFNN_V2(in_channel,9,1,resname)
# model = EncoderCNN(in_channel, 1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)


criterion = torch.nn.MSELoss()  # Mean Squared Error loss function
# optimizer = optim.Adam(list(conv.parameters()) + list(deconv.parameters()), lr=0.001)  # Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer



In [ ]:

model = train_with_cross_validation(model, train_val_dataset, batch_size, num_epochs, optimizer, criterion, is_dual_data=True)


# model_name = "path/model_img(nbands="+str(in_channel)+")_metadata_" + crop_var[analyze_variety_id]+"_"+ model.__class__.__name__+ img_path[-24:-17] +resname+"__Batch=" +str(batch_size) + "_state.pth"
model_name = "path/model_img(nbands="+str(in_channel)+")_metadata_" + crop_var[analyze_variety_id]+"_"+ model.__class__.__name__+ '_' +resname+"__Batch=" +str(batch_size) + "_state.pth"

torch.save(model.state_dict(), model_name)


In [ ]:
# model_name = "../output/path/model_metadata_ResNetFNN_DOY186_Batch=32_state.pth"
# # in_channel = 5
# # num_epochs = 200
# # batch_size = 32

# loaded_model = ResNetFNN(in_channel,9,1)
# loaded_model.load_state_dict(torch.load(model_name))
# loaded_model.to(device)

In [ ]:
test_accuracy, test_prediction = validate(model, test_dataset,criterion, batch_size = batch_size, is_return_output = True, is_dual_data=True)
test_accuracy = np.sqrt(np.mean(test_accuracy))
print(f'validation rmse is {test_accuracy}')

In [ ]:
from ml.ml_predict import plot_result_separate
yield_data = np.array(yield_list)
irrigate_data = np.array(yield_pf['Irrigation_int'])

test_truth = yield_data[test_indices]


name_tag = img_path[-23:-17]
# name_tag = 'ALL Data'
out_name = name_tag + ' ' + crop_var[analyze_variety_id] + ' ' 
out_name = out_name + keyword[:-11] 
out_name = out_name + ''

plot_result_separate(np.array(test_truth), np.array(test_prediction), test_indices, irrigate_data, out_name)

In [ ]:

def plot_result_separate(y_test, y_pred, test_indices, irrigate_data,save_name):
    
    basename = os.path.basename(save_name)
    
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    x = np.arange(50,300)
    plt.plot(x, x, color = 'k', ls='--', alpha=0.7, linewidth=1.25)
    
    deficit_indices = np.where(irrigate_data == 1)[0]
    full_indices = np.where(irrigate_data == 0)[0]
    # plt.scatter( y_test, y_pred, label = f'r-squared = {round(r2,2)}, rmse = {round(rmse, 2)}')
    plt.scatter( y_test[np.in1d(test_indices, deficit_indices)], 
                y_pred[np.in1d(test_indices, deficit_indices)],
                marker='x', s=25, color = '#bcbd22', label = 'Deficit irrigated')
    plt.scatter( y_test[np.in1d(test_indices, full_indices)], 
                y_pred[np.in1d(test_indices, full_indices)],
                label = 'Fully irrigated', 
                marker='o', alpha=0.5, s=25, color = '#2ca02c', edgecolors='#2ca02c')
    # , facecolor='none'
    
    plt.xlim([50,300])
    plt.ylim([50,300])
    plt.xticks(fontname='Times New Roman', size=10)
    plt.yticks(fontname='Times New Roman', size=10)
    plt.xlabel('True Yield (Bu/Ac)', fontname = 'Times New Roman', fontsize = 12)
    plt.ylabel('Predicted Yield (Bu/Ac)', fontname = 'Times New Roman', fontsize = 12)

    plt.title(basename, fontname = 'Times New Roman', fontsize = 14)

    plt.text(55, 288, f'R-squared = {round(r2,2)}', 
             fontname = 'Times New Roman', fontsize = 11)
    plt.text(55, 275, f'RMSE = {round(rmse, 2)} (Bu/Ac)',  
             fontname = 'Times New Roman', fontsize = 11)
    
    font_props = FontProperties(family='Times New Roman', size=11)
    plt.legend(loc='lower right', frameon=False, 
               prop=font_props)
    # yy_pred = model_lasso.predict(X_train)
    # plt.scatter(y_train, yy_pred)
    plt.savefig(save_name + '.png', dpi=300, bbox_inches='tight')
    # plt.show()
    plt.close()
    
    print(save_name, f' r-squared = {r2}, rmse = {rmse}, mae={mae}')
    
    

In [ ]:
# def plot_result_metadata():